# import 및 데이터불러오기

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 서울시 구별전월세전환율
df_guchange=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 지역별 전월세전환율(아파트).csv', encoding='cp949')

# 서울시 전월세가(20년-22년)
df_home20=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/20년 서울시 전월세가.csv', encoding='cp949')
df_home21=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/21년 서울시 전월세가.csv', encoding='cp949')
df_home22=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/22년 서울시 전월세가.csv', encoding='cp949')

<ipython-input-3-4654336d5344>:5: DtypeWarning: Columns (6,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_home20=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/20년 서울시 전월세가.csv', encoding='cp949')
<ipython-input-3-4654336d5344>:6: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_home21=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/21년 서울시 전월세가.csv', encoding='cp949')


# 전월세 전환율

In [ ]:
# 동별 전환율
df_guchange['지역'] = df_guchange['지역'].str.split(' ').str[1]
df_guchange.set_index('지역', inplace=True)
df_guchange=df_guchange.iloc[13:38]
df_guchange = df_guchange[['2023-04']]
df_guchange.rename(columns={'2023-04':'구별 전월세전환율'}, inplace=True)

# 4대 은행 전환율
df_guchange['신한'] = 4.37
df_guchange['국민'] = 4.58
df_guchange['우리'] = 4.66
df_guchange['하나'] = 4.37

# 법정 전환율
df_guchange['법정금리'] = 5.5

# 평균 전환율
df_guchange['평균금리'] = df_guchange.mean(axis='columns')
df_guchange=df_guchange[['평균금리']]
df_guchange.head()

,평균금리
지역,
종로구,4.713333
중구,4.763333
용산구,4.763333
성동구,4.796667
광진구,4.713333


# 전월세 데이터, 전월세 전환율 merge

In [ ]:
# 20~22년 merge
df_home=pd.concat([df_home20, df_home21])
df_home=pd.concat([df_home, df_home22])

# 주택 연식 계산
df_home['연식']=df_home['계약일'].astype(str).str[:4].astype(int) - df_home['건축년도']

# 전월세전환율, 전월세 데이터 merge
df_home=pd.merge(df_home, df_guchange, left_on='자치구명', right_on='지역', how='outer')
df_home=df_home[['자치구명', '법정동명', '층', '계약일', '임대면적', '보증금(만원)', '임대료(만원)', '건물명', '건물용도', '연식', '평균금리']]
df_home

,자치구명,법정동명,층,계약일,임대면적,보증금(만원),임대료(만원),건물명,건물용도,연식,평균금리
0,용산구,원효로2가,6.0,20200810,22.03,5000,0,\t(1)\t,오피스텔,30.0,4.763333
1,용산구,한남동,1.0,20200319,68.87,36000,0,\t(726-116)\t,오피스텔,18.0,4.763333
2,용산구,한남동,1.0,20200824,68.87,36000,0,\t(726-116)\t,오피스텔,18.0,4.763333
3,용산구,용산동2가,2.0,20180305,48.41,1000,60,(1-1131),연립다세대,18.0,4.763333
4,용산구,용산동2가,1.0,20200610,48.41,25000,0,(1-1131),연립다세대,20.0,4.763333
...,...,...,...,...,...,...,...,...,...,...,...
1594373,강북구,번동,16.0,20221231,117.28,45000,0,오동공원현대,아파트,21.0,4.813333
1594374,강북구,우이동,1.0,20221231,76.87,20000,0,상아블루맨션A,연립다세대,20.0,4.813333
1594375,강북구,미아동,3.0,20221231,84.58,37065,0,미아1차래미안,아파트,16.0,4.813333
1594376,강북구,미아동,17.0,20221231,59.98,43000,0,에스케이북한산시티,아파트,18.0,4.813333


# 데이터 전처리

## 층 수

In [ ]:
# 층수 결측치(단독다가구 nan값) 및 지하(-1~-4) 처리(-1층)
for idx in range(len(df_home)):
    if df_home.loc[idx, '건물용도']=='단독다가구':
      df_home.loc[idx, '층'] = 1
    elif df_home.loc[idx, '층'] < 0:
      df_home.loc[idx, '층'] = -1

## 연식

In [ ]:
# 연식 100년 이상, -3년미만 drop(이상치로 판단)
df_home.loc[df_home['연식']>=100].index
df_home.loc[df_home['연식']<-3].index
                      # 연식 100 이상 drop
df_home.drop(labels=[22399, 25094, 38414, 100257, 117697, 118083, 171068, 552648, 566770, 569612, 573558, 882078, 1007327,
                      1127702, 1187708, 1195574, 1196656, 1201372, 1217090, 1247739, 1280016, 1437334, 1562504,
                      # 연식 -4 이하 drop
                      207929, 207930, 208300, 341856, 341996, 353415, 465742, 799303, 847950, 907036, 918948, 967547,
                      967552, 967588, 975274, 980754, 981815, 986943, 993547, 997181, 1529093],
             axis=0, inplace=True)

# 연식 -3 ~ -1은 0으로 처리(신식)
for idx in range(len(df_home)):
    if df_home.loc[idx, '연식'] == -1 or df_home.loc[idx, '연식'] == -2 or df_home.loc[idx, '연식'] == -3:
      df_home.loc[idx, '연식'] = 0

df_home.reset_index(drop=True, inplace=True)
df_home

,자치구명,법정동명,층,계약일,임대면적,보증금(만원),임대료(만원),건물명,건물용도,연식,평균금리
0,용산구,원효로2가,6.0,20200810,22.03,5000,0,\t(1)\t,오피스텔,30.0,4.763333
1,용산구,한남동,1.0,20200319,68.87,36000,0,\t(726-116)\t,오피스텔,18.0,4.763333
2,용산구,한남동,1.0,20200824,68.87,36000,0,\t(726-116)\t,오피스텔,18.0,4.763333
3,용산구,용산동2가,2.0,20180305,48.41,1000,60,(1-1131),연립다세대,18.0,4.763333
4,용산구,용산동2가,1.0,20200610,48.41,25000,0,(1-1131),연립다세대,20.0,4.763333
...,...,...,...,...,...,...,...,...,...,...,...
1594329,강북구,번동,16.0,20221231,117.28,45000,0,오동공원현대,아파트,21.0,4.813333
1594330,강북구,우이동,1.0,20221231,76.87,20000,0,상아블루맨션A,연립다세대,20.0,4.813333
1594331,강북구,미아동,3.0,20221231,84.58,37065,0,미아1차래미안,아파트,16.0,4.813333
1594332,강북구,미아동,17.0,20221231,59.98,43000,0,에스케이북한산시티,아파트,18.0,4.813333


In [ ]:
# 연식 결측치 갯수(약 28000개)
df_home.loc[df_home['연식'].isnull()]

,자치구명,법정동명,층,계약일,임대면적,보증금(만원),임대료(만원),건물명,건물용도,연식,평균금리
2331,용산구,한강로2가,16.0,20200108,42.50,2000,115,래미안 용산 더 센트럴,오피스텔,NaN,4.763333
2332,용산구,한강로2가,10.0,20200923,42.50,42000,0,래미안 용산 더 센트럴,오피스텔,NaN,4.763333
2333,용산구,한강로2가,6.0,20181212,47.65,2500,140,래미안 용산 더 센트럴,오피스텔,NaN,4.763333
2334,용산구,한강로2가,16.0,20200229,47.65,32000,0,래미안 용산 더 센트럴,오피스텔,NaN,4.763333
2335,용산구,한강로2가,6.0,20200429,47.65,46000,0,래미안 용산 더 센트럴,오피스텔,NaN,4.763333
...,...,...,...,...,...,...,...,...,...,...,...
1593791,강북구,우이동,1.0,20221128,30.02,3304,7,NaN,단독다가구,NaN,4.813333
1593983,강북구,미아동,1.0,20221209,62.81,4000,20,NaN,단독다가구,NaN,4.813333
1594010,강북구,수유동,1.0,20221210,22.00,4500,0,NaN,단독다가구,NaN,4.813333
1594178,강북구,수유동,1.0,20221220,33.00,500,30,NaN,단독다가구,NaN,4.813333


## 전월세 전환

In [ ]:
# 전월세 전환
df_home['전세가'] = 0
for idx in range(len(df_home)):
    월세 = df_home.loc[idx, '임대료(만원)']
    전월세전환율 = df_home.loc[idx, '평균금리']
    월세보증금 = df_home.loc[idx, '보증금(만원)']
    if 월세 == 0:
        df_home.loc[idx, '전세가'] = df_home.loc[idx, '보증금(만원)']
    else:
        df_home.loc[idx, '전세가'] = (월세 * 12 * 100) / 전월세전환율 + 월세보증금
df_home.drop(['보증금(만원)', '임대료(만원)', '평균금리'], axis=1, inplace=True)
df_home.head()

,자치구명,법정동명,층,계약일,임대면적,건물명,건물용도,연식,전세가
0,용산구,원효로2가,6.0,20200810,22.03,\t(1)\t,오피스텔,30.0,5000.00000
1,용산구,한남동,1.0,20200319,68.87,\t(726-116)\t,오피스텔,18.0,36000.00000
2,용산구,한남동,1.0,20200824,68.87,\t(726-116)\t,오피스텔,18.0,36000.00000
3,용산구,용산동2가,2.0,20180305,48.41,(1-1131),연립다세대,18.0,16115.46536
4,용산구,용산동2가,1.0,20200610,48.41,(1-1131),연립다세대,20.0,25000.00000


## 평당 전세가

In [ ]:
# 1평 = 3.3m^2
df_home['평당전세가']=df_home['전세가']/(df_home['임대면적'] / 3.3)
df_home

,자치구명,법정동명,층,계약일,임대면적,건물명,건물용도,연식,전세가,평당전세가
0,용산구,원효로2가,6.0,20200810,22.03,\t(1)\t,오피스텔,30.0,5000.00000,748.978665
1,용산구,한남동,1.0,20200319,68.87,\t(726-116)\t,오피스텔,18.0,36000.00000,1724.989110
2,용산구,한남동,1.0,20200824,68.87,\t(726-116)\t,오피스텔,18.0,36000.00000,1724.989110
3,용산구,용산동2가,2.0,20180305,48.41,(1-1131),연립다세대,18.0,16115.46536,1098.554755
4,용산구,용산동2가,1.0,20200610,48.41,(1-1131),연립다세대,20.0,25000.00000,1704.193348
...,...,...,...,...,...,...,...,...,...,...
1594329,강북구,번동,16.0,20221231,117.28,오동공원현대,아파트,21.0,45000.00000,1266.200546
1594330,강북구,우이동,1.0,20221231,76.87,상아블루맨션A,연립다세대,20.0,20000.00000,858.592429
1594331,강북구,미아동,3.0,20221231,84.58,미아1차래미안,아파트,16.0,37065.00000,1446.139749
1594332,강북구,미아동,17.0,20221231,59.98,에스케이북한산시티,아파트,18.0,43000.00000,2365.788596


# 연식 결측치 평균대체

In [ ]:
df_home1 = df_home.copy()

# 연식 NaN값 = 건축년도 NaN값인 경우 방법 1
# 동, 건물용도 group화해서 평균
# 동 group화해서 평균
# 구, 건물용도 group화 해서 평균 3단계로 진행
df_home1['연식'].fillna(df_home1.groupby(['법정동명', '건물용도'])['연식'].transform('mean'), inplace=True)
df_home1['연식'].fillna(df_home1.groupby('법정동명')['연식'].transform('mean'), inplace=True)
df_home1['연식'].fillna(df_home1.groupby(['자치구명', '건물용도'])['연식'].transform('mean'), inplace=True)

df_home1.iloc[2331:2335]

,자치구명,법정동명,층,계약일,임대면적,보증금(만원),임대료(만원),건물명,건물용도,연식,평균금리
2331,용산구,한강로2가,16.0,20200108,42.50,2000,115,래미안 용산 더 센트럴,오피스텔,9.946625,4.763333
2332,용산구,한강로2가,10.0,20200923,42.50,42000,0,래미안 용산 더 센트럴,오피스텔,9.946625,4.763333
2333,용산구,한강로2가,6.0,20181212,47.65,2500,140,래미안 용산 더 센트럴,오피스텔,9.946625,4.763333
2334,용산구,한강로2가,16.0,20200229,47.65,32000,0,래미안 용산 더 센트럴,오피스텔,9.946625,4.763333


In [ ]:
df_home1.to_csv("/content/drive/MyDrive/뽀로로드/data(아파트시세_코랩가공)/서울시 전세(연식 결측치 대체).csv", encoding='cp949')

# 연식 결측치 대체 X, 연식 평균 컬럼 추가

In [ ]:
df_home2 = df_home.copy()

# 연식 NaN값 = 건축년도 NaN값인 경우 방법 2
df_home2['동별 평균 연식'] = np.nan
df_home2['구별 평균 연식'] = np.nan

df_home2['동별 평균 연식'].fillna(df_home2.groupby('법정동명')['연식'].transform('mean'), inplace=True)
df_home2['구별 평균 연식'].fillna(df_home2.groupby('자치구명')['연식'].transform('mean'), inplace=True)

df_home2.iloc[2331:2335]

,자치구명,법정동명,층,계약일,임대면적,보증금(만원),임대료(만원),건물명,건물용도,연식,평균금리,동별 평균 연식,구별 평균 연식
2331,용산구,한강로2가,16.0,20200108,42.50,2000,115,래미안 용산 더 센트럴,오피스텔,NaN,4.763333,6.883032,20.914235
2332,용산구,한강로2가,10.0,20200923,42.50,42000,0,래미안 용산 더 센트럴,오피스텔,NaN,4.763333,6.883032,20.914235
2333,용산구,한강로2가,6.0,20181212,47.65,2500,140,래미안 용산 더 센트럴,오피스텔,NaN,4.763333,6.883032,20.914235
2334,용산구,한강로2가,16.0,20200229,47.65,32000,0,래미안 용산 더 센트럴,오피스텔,NaN,4.763333,6.883032,20.914235


In [ ]:
df_home2.to_csv("/content/drive/MyDrive/뽀로로드/data(아파트시세_코랩가공)/서울시 전세(연식 평균 컬럼 추가).csv", encoding='cp949')